<a href="https://colab.research.google.com/github/jun-wei-lin/NCHU/blob/main/AIoT-DA/Transfer%20Learning%20and%20Hugging%20Face/HW6_1_Transfer_learning_on_pretrained_VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

步驟 1：克隆資料集

In [30]:
# 在 Colab 中執行以下程式碼來克隆資料庫
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git

fatal: destination path 'Face-Mask-Detection' already exists and is not an empty directory.


步驟 2：設置資料路徑與載入資料

In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 資料路徑
dataset_dir = 'Face-Mask-Detection/dataset'

# 數據生成器
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)


Found 3274 images belonging to 2 classes.
Found 818 images belonging to 2 classes.


步驟 3：建構 VGG16 遷移學習模型

In [32]:
# 建構 VGG16 模型
def build_vgg16_model(num_classes=2):
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_vgg16_model(num_classes=2)

步驟 4：訓練模型

In [33]:
# 訓練模型
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.8609 - loss: 0.3816 - val_accuracy: 0.9841 - val_loss: 0.0493
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.9760 - loss: 0.0697 - val_accuracy: 0.9756 - val_loss: 0.0725
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9833 - loss: 0.0479 - val_accuracy: 0.9841 - val_loss: 0.0415
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 20s 54ms/step - accuracy: 0.9837 - loss: 0.0415 - val_accuracy: 0.9817 - val_loss: 0.0569
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9874 - loss: 0.0339 - val_accuracy: 0.9866 - val_loss: 0.0414
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9770 - loss: 0.0517 - val_accuracy: 0.9670 - val_loss: 0.0981
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - accuracy: 0.9896 - loss: 0.0252 - val_accuracy: 0.9841 - val_loss: 0.0507
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.9906 - loss: 0.0246 - 

步驟 5：測試模型

In [34]:
# 評估模型
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {test_loss:.4f}, Validation Accuracy: {test_accuracy:.4f}")

52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9848 - loss: 0.0482
Validation Loss: 0.0647, Validation Accuracy: 0.9817


步驟 6：從圖片網址進行分類

In [38]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import requests
from PIL import Image
from io import BytesIO

# 處理圖片的函數
def preprocess_image_from_url(image_url, target_size=(128, 128)):
    response = requests.get(image_url)  # 下載圖片
    image = Image.open(BytesIO(response.content))  # 開啟圖片
    image = image.convert('RGB')  # 確保是 RGB 模式
    image = image.resize(target_size)  # 調整大小
    img_array = img_to_array(image) / 255.0  # 正規化
    return np.expand_dims(img_array, axis=0)  # 增加批次維度

# 測試圖片分類的函數
def test_image(image_url, model, class_indices):
    # 將 class_indices 的鍵值對轉換成索引到類別名稱的映射
    idx_to_class = {v: k for k, v in class_indices.items()}

    # 預處理輸入圖片
    img_array = preprocess_image_from_url(image_url)

    # 預測
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions)  # 找到預測的索引
    predicted_class = idx_to_class[predicted_class_idx]  # 索引映射到類別名稱

    print(f"該圖片分類結果為：{predicted_class}")

# 使用範例
image_url = input("輸入圖片網址: ")
test_image(image_url, model, train_generator.class_indices)


輸入圖片網址: https://github.com/chandrikadeb7/Face-Mask-Detection/blob/master/dataset/without_mask/0_0_aidai_0084.jpg?raw=true
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
該圖片分類結果為：without_mask
